# Word Embeddings : le modèle Word2Vec

## Imports

In [3]:
import sys
import os
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

### telecharger les stops words

In [4]:
# Télécharger stopwords
nltk.download('stopwords')
sw = set(stopwords.words("french"))

# Stopwords supplémentaires
extra_sw = {
    "les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
    "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
    "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
    "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
    "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "etxc", "elles", "dés",
    "prixx", "écr", "géné", "app", "adr", "mod", "bur", "trav", "et", "de", "à", "en",
    "le", "la", "du", "des", "aux", "un", "une"
}
sw |= extra_sw

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilaria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Fonction nettoyage de texte

In [5]:
def clean_text_string(text):
    # Supprimer caractères non alphabétiques
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", " ", text)
    # Enlever les répétitions de lettres aberrantes (ex: iiiii, llll)
    text = re.sub(r"(.)\1{2,}", r"\1", text)
    # Minuscules
    text = text.lower()
    # Tokenisation simple
    words = nltk.wordpunct_tokenize(text)
    # Filtrage avancé
    kept = [
        w for w in words
        if (
            len(w) > 2 and           # minimum 3 lettres
            len(w) < 20 and          # éviter les tokens trop longs ("ltshdyyysnkjdlqj")
            w.isalpha() and
            w not in sw and
            not w.startswith(("ij", "vv", "qq")) and  # motifs OCR fréquents
            not re.match(r".*[0-9].*", w)             # pas de chiffres
        )
    ]
    return " ".join(kept)


### Lecture du texte nettoyé

In [6]:
# Chemin vers ton fichier
infile = "../../data/sents.txt"

# Lecture de toutes les lignes
with open(infile, encoding="utf-8") as f:
    texts = f.readlines()

# Nettoyage
cleaned_texts = [clean_text_string(text) for text in texts]
# On enlève les lignes vides après nettoyage
cleaned_texts = [t for t in cleaned_texts if t.strip() != ""]

print(f"Nombre de phrases nettoyées : {len(cleaned_texts)}")
print(cleaned_texts[:5])  # afficher les 5 premières phrases pour vérification


Nombre de phrases nettoyées : 891576
['imnri hmu marché tenu hors villa', 'vaain téicj races indigènes', 'rasa îichakdui taureaux iallsènas', 'hollandais dufr', 'îdto vachei laitières vante']


## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [7]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [8]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

In [9]:
import os
print(os.getcwd())


/Users/ilaria/Desktop/STIC/traitement_auto_corpus/tac/tps/tp3


### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [11]:
bigram_phrases = Phrases(sentences)

In [12]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [13]:
len(bigram_phrases.vocab.keys())

4310706

Prenons une clé au hasard :

In [14]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [15]:
bigram_phrases.vocab[key_]

42

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [16]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [17]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [18]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [19]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [20]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [21]:
# Liste de combinaisons de paramètres à tester
params_list = [
    {"vector_size": 50, "window": 5, "min_count": 5},
    {"vector_size": 100, "window": 10, "min_count": 5},
    {"vector_size": 100, "window": 5, "min_count": 10},
]

models = {}

for i, params in enumerate(params_list):
    print(f"\n Entraînement du modèle {i+1} avec paramètres : {params}")
    model = Word2Vec(
        corpus,
        vector_size=params["vector_size"],
        window=params["window"],
        min_count=params["min_count"],
        workers=4,
        epochs=5
    )
    models[f"model_{i+1}"] = model
    model.save(f"../../data/word2vec_model_{i+1}.model")
    print(f" Modèle {i+1} sauvegardé !")



 Entraînement du modèle 1 avec paramètres : {'vector_size': 50, 'window': 5, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 1 sauvegardé !

 Entraînement du modèle 2 avec paramètres : {'vector_size': 100, 'window': 10, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 2 sauvegardé !

 Entraînement du modèle 3 avec paramètres : {'vector_size': 100, 'window': 5, 'min_count': 10}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 3 sauvegardé !


## Exploration et comparaison des modèles

In [22]:
# Charger et explorer les modèles entraînés
for name, model in models.items():
    print(f"\n=== Exploration de {name} ===")

    # 3 exemples de similarité
    pairs = [("ministre", "roi"), ("homme", "femme"), ("paris", "londres")]
    for w1, w2 in pairs:
        try:
            sim = model.wv.similarity(w1, w2)
            print(f"Similarité({w1}, {w2}) = {sim:.3f}")
        except KeyError:
            print(f"{w1} ou {w2} absent du vocabulaire.")

    # 3 exemples de recherche de mots proches
    terms = ["ministre", "guerre", "belgique"]
    for t in terms:
        try:
            print(f"\nMots les plus proches de '{t}' :")
            for word, score in model.wv.most_similar(t, topn=5):
                print(f"  {word:20s} {score:.3f}")
        except KeyError:
            print(f"{t} absent du vocabulaire.")



=== Exploration de model_1 ===
Similarité(ministre, roi) = 0.453
Similarité(homme, femme) = 0.649
Similarité(paris, londres) = 0.857

Mots les plus proches de 'ministre' :
  secretaire           0.855
  adjoint              0.814
  membre               0.814
  president            0.814
  en_remplacement      0.797

Mots les plus proches de 'guerre' :
  la_compagnie         0.824
  pekin                0.818
  retraite             0.802
  la_defense           0.798
  les_membres          0.786

Mots les plus proches de 'belgique' :
  suisse               0.788
  francfort            0.764
  la_banque            0.763
  medecine             0.762
  hollande             0.755

=== Exploration de model_2 ===
Similarité(ministre, roi) = 0.507
Similarité(homme, femme) = 0.567
Similarité(paris, londres) = 0.780

Mots les plus proches de 'ministre' :
  secretaire           0.840
  secretaire_general   0.803
  president            0.803
  vice_-_president     0.793
  conseiller           0.79

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [23]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [24]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [25]:
model.wv["ministre"]
#moins c'est moins proche, plus c'est plur proche

array([ 1.02499294e+00,  3.82762738e-02, -8.93529475e-01,  5.47164679e-02,
        8.22362840e-01,  2.10928822e+00, -1.01254009e-01, -1.20475852e+00,
       -2.56319204e-03,  9.00391281e-01,  8.46700251e-01,  1.33263206e+00,
       -6.70013130e-01, -4.77592051e-01, -7.70986155e-02, -1.15629065e+00,
        1.88836098e+00,  1.48250294e+00, -1.07860732e+00, -2.31762815e+00,
       -1.84084281e-01, -1.25240505e+00,  1.09978080e+00,  5.17830670e-01,
       -1.01580024e+00,  1.04826128e+00, -4.53097880e-01,  1.65025902e+00,
        1.84959427e-01,  1.40506566e+00, -1.50112414e+00, -4.30370748e-01,
        1.90948701e+00, -2.03920007e+00, -8.02095771e-01,  1.41926026e+00,
        5.75048566e-01, -9.01740551e-01, -4.59442347e-01, -1.56654096e+00,
        7.43895322e-02, -1.87444532e+00,  3.51592004e-01,  7.43975863e-02,
       -5.06704509e-01,  5.59601545e-01,  9.45686162e-01, -1.95080650e+00,
       -2.91851223e-01, -3.98556888e-02,  8.02248716e-01, -1.46690166e+00,
       -6.39869213e-01,  

### Calculer la similarité entre deux termes

In [26]:
model.wv.similarity("ministre", "roi")

np.float32(0.5166741)

### Chercher les mots les plus proches d'un terme donné

In [27]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.8270032405853271),
 ('president', 0.7864551544189453),
 ('membre', 0.7797642350196838),
 ('vice_-_president', 0.7794414758682251),
 ('au_ministere', 0.7696550488471985),
 ('rapporteur', 0.7654617428779602),
 ('ancien_ministre', 0.7648983597755432),
 ('depute', 0.7584162354469299),
 ('secretaire_general', 0.7553635835647583),
 ('adjoint', 0.7516523599624634)]

### Faire des recherches complexes à travers l'espace vectoriel

In [28]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))
#on retire le concept belgique aux concept paris et londre, 

[('berlin', 0.6857337951660156), ('la_monnaie', 0.6483603715896606), ('marseille', 0.6317960023880005), ('ce_matin', 0.6298128962516785), ('new_-_york', 0.6292555332183838), ('nice', 0.6158302426338196), ('au_theatre', 0.5965142250061035), ('washington', 0.5771775841712952), ('dimanche', 0.5762718319892883), ('hier_soir', 0.5736894011497498)]
